# Projekt: Rozpoznanie liter

## Opis zadania
Celem jest klasyfikacja 20 000 liter na podstawie 16 atrybutów numerycznych. Pierwsza kolumna pliku `letter-recognition.data` zawiera etykietę (literę), a kolejne 16 kolumn to cechy.

## Metoda
- Normalizacja cech do zakresu (0, 1) przy użyciu `MinMaxScaler`.
- Sztywny podział danych bez mieszania: pierwsze 16 000 próbek do treningu, pozostałe 4 000 do testu.
- Modele: k-NN oraz Gaussian Naive Bayes.


## Instalacja bibliotek
Rekomendowane jest własne środowisko (venv), żeby uniknąć błędu PEP 668:

```
python3 -m venv .venv
source .venv/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install pandas scikit-learn
```

Po instalacji wybierz kernel z `.venv` w Jupyter.


In [ ]:
# Szybka weryfikacja środowiska (uruchom po instalacji)
import sys
import subprocess

try:
    import sklearn  # noqa: F401
except ModuleNotFoundError:
    subprocess.check_call(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "pandas",
            "scikit-learn",
        ]
    )
    import sklearn  # noqa: F401

print("OK: scikit-learn jest dostępny w kernelu:", sys.executable)


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

# Wczytanie danych bez nagłówka
# Pierwsza kolumna: etykieta, kolejne 16: cechy

data = pd.read_csv(
    "/Users/mateuszgorka/Downloads/letter+recognition/letter-recognition.data",
    header=None,
)

y = data.iloc[:, 0]
X = data.iloc[:, 1:]


: 

In [ ]:
# Normalizacja cech do (0, 1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Sztywny podział: 16 000 trening, 4 000 test
X_train, X_test = X_scaled[:16000], X_scaled[16000:]
y_train, y_test = y.iloc[:16000], y.iloc[16000:]


In [ ]:
# Widoczny podział na zbiory
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)


In [ ]:
# Porównanie kilku ustawień k-NN
knn_results = []

for n_neighbors in [1, 3, 5, 7, 9, 11]:
    for weights in ["uniform", "distance"]:
        for metric in ["euclidean", "manhattan"]:
            model = KNeighborsClassifier(
                n_neighbors=n_neighbors,
                weights=weights,
                metric=metric,
            )
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            acc = accuracy_score(y_test, preds)
            knn_results.append(
                {
                    "Model": "k-NN",
                    "Parametry": f"k={n_neighbors}, wagi={weights}, metryka={metric}",
                    "Accuracy": acc,
                }
            )

# Porównanie kilku ustawień GaussianNB
nb_results = []

for var_smoothing in [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]:
    model = GaussianNB(var_smoothing=var_smoothing)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    nb_results.append(
        {
            "Model": "Naive Bayes",
            "Parametry": f"var_smoothing={var_smoothing:g}",
            "Accuracy": acc,
        }
    )

knn_results_df = pd.DataFrame(knn_results).sort_values(
    by="Accuracy", ascending=False
)
nb_results_df = pd.DataFrame(nb_results).sort_values(by="Accuracy", ascending=False)

best_knn = knn_results_df.iloc[0]
best_nb = nb_results_df.iloc[0]

best_overall = pd.concat([best_knn.to_frame().T, best_nb.to_frame().T]).sort_values(
    by="Accuracy", ascending=False
).iloc[0]

print(
    f"Najlepszy k-NN: {best_knn['Accuracy']:.4f} ({best_knn['Parametry']})"
)
print(
    f"Najlepszy Naive Bayes: {best_nb['Accuracy']:.4f} ({best_nb['Parametry']})"
)


In [ ]:
# Tabele wyników (top 5 dla każdego modelu)
knn_results_df.head(5)

nb_results_df.head(5)


In [ ]:
# Wnioski: najlepsze ustawienie dla każdego modelu i zwycięzca
if best_overall["Model"] == "k-NN":
    better_model = "k-NN"
else:
    better_model = "Naive Bayes"

print(f"Lepszy model: {better_model}")
print(
    f"Najlepsze parametry: {best_overall['Parametry']} (Accuracy={best_overall['Accuracy']:.4f})"
)


## Wyniki
W tabelach zestawione są różne konfiguracje k-NN i GaussianNB, co pozwala łatwo wychwycić, które ustawienia wypadają najlepiej.

## Wnioski
Na końcu notatnika pojawia się krótka informacja o najlepszym wariancie oraz zwycięskim modelu na zbiorze testowym.
